# Final Project

## Part 1: Data Collection 

The early youth of a child is a developmental time where students are learning how to perform many tasks and learn skills, both book smart and street smart, that can help them in life. One of those skills that begins to develop in a young age is literacy in basic math and reading, as the majority of math that one deals with in adulthood is taught in middle school (get reference), and reading comprehension is key to understanding the majority of events that happen in an adults life - understanding forms, learning new information, searching for housing, etc. Therefore, it is important that all children in this developmental stage have equitable opportunities deserving of them that in such a key growth period, they all have the tools and education necessary to learn such important and long lasting skills such as math and reading comprehension.

However, not all students are given such equally fitted opportunities. The US education system has long been known to have varying standards of education (GET REFERENCE), where differences in education quality begin as early as pre-kindergarten, but not a lot of documentation has been procured to confirm on any large variation in education quality. It is imperative that if these differences in education quality exist, then they be resolved on an institutional level. 

So, our focus of project is to confirm if education inequality is reflected by national math and reading examination differences and recognize factors such as race or gender or state that may play significant roles in such (if they exist), and use such analysis to predict how future years education inequality will be if the current education system/institution is maintained. 

Our null hypothesis will be that race, gender, and state do not have any relationship or impact on math or reading literacy in children in developmental stages. Our alternative hypothesis will be that race, gender, and state have some relationship or impact on math or reading literacy in children in developmental stages.

## Part 2: Data Management/Representation

First we have to import the necessary libraries that we need to load the dataset. We are using pandas, numpy, and matplotlib.pyplot. Pandas is used for the DataFrame object since that is an easy way to store tabular data. Numpy is used for its math functionality and mathplotlib.pyplot is used to plot graphs demonstrating relationships between variables in our data.

Since the data is in a csv file in the directory, we need to load it into a pandas DataFrame so that we can easily manipulate it.

Now we have to clean the data. I am first going to drop columns that we do not need (ex: EventId, StartTime, etc). Then we rearranged and renamed the columns so that it was easier to read the dataset.

Now to deal with any missing variables, we are dropping any rows that have NaN or "" values since those rows cannot be used to make our prediction and so it would be easier for us to just not have them. We also have enough data to supplement the values that are going to be lost by dropping rows with missing values.

## Exploratory Data Analysis

## Hypothesis testing

## Communication of Insights Attained